In [17]:
%reset -f

import os
import numpy as np
import numba as  nb
import pandas as pd


import plotly.graph_objects as go
from plotly.graph_objects import Figure

from plotly.graph_objects import Scatter
from plotly.graph_objects import Ohlc

from sxo.util.binning import bin_values
from sxo.util.runtime.timeseries import RedisTs

In [18]:
os.environ["REDIS_PASS"] = "boss"

In [19]:
ts_bid = RedisTs('ts_bid_FxSpot_GBPEUR')
ts_ask = RedisTs('ts_ask_FxSpot_GBPEUR')
ts_bsz = RedisTs('ts_bsz_FxSpot_GBPEUR')
ts_asz = RedisTs('ts_asz_FxSpot_GBPEUR')
bid = ts_bid.get_range(name='bid')
ask = ts_ask.get_range(name='ask')
bsz = ts_bid.get_range(name='bsz')
asz = ts_ask.get_range(name='asz')
bid_ask = bid.join(ask, how='inner').join(bsz, how='inner').join(asz, how='inner').reset_index()
bid_ask['t'] = bid_ask['t'].values.astype('datetime64[ms]').astype('datetime64[ns]')


In [20]:
bid_ask['mid'] = (bid_ask['bid'].values + bid_ask['ask'].values) / 2

In [21]:
bid_ask.to_parquet('tmp.parq')
#bid_ask = pd.read_parquet('tmp.parq')

In [22]:

bins = bin_values(bid_ask, 60, time_col='t', value_col='mid')
bins.tail(4)

,bin_start,bin_end,t0,t1,n_samples,o,h,l,c,twap
43,2023-10-02 19:56:00,2023-10-02 19:57:00,2023-10-02 19:56:00.733,2023-10-02 19:56:57.442,31,1.153645,1.153725,1.153645,1.153695,1.153685
44,2023-10-02 19:57:00,2023-10-02 19:58:00,2023-10-02 19:57:03.034,2023-10-02 19:57:59.440,27,1.153665,1.153705,1.153655,1.153705,1.153691
45,2023-10-02 19:58:00,2023-10-02 19:59:00,2023-10-02 19:58:00.391,2023-10-02 19:58:58.877,43,1.153685,1.153715,1.153595,1.153615,1.153676
46,2023-10-02 19:59:00,2023-10-02 20:00:00,2023-10-02 19:59:00.241,2023-10-02 19:59:57.990,38,1.153605,1.153765,1.153585,1.153745,1.153644


In [16]:
import datetime as dt

df_plt = bid_ask.copy()[bid_ask.t > np.datetime64('2023-07-06 06:00:00')]
bin_plt = bins.copy()[bins.bin_start > np.datetime64('2023-07-06 12:00:00')]
m = 15
# fig = Figure(layout=go.Layout(height=500, margin={'t':m,'b':m,'l':m,'r':m}))
# fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':2},showlegend=False))
# fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':2},showlegend=False))
#fig.add_traces(Scatter(x=bin_plt['bin_start'], y=bin_plt['twap'], name='bid', showlegend=False))

# s, e, h, l  = bin_plt['bin_start'].values.astype(pd.Timestamp), bin_plt['bin_end'].values.astype(pd.Timestamp), bin_plt['h'].values, bin_plt['l'].values
# i = 3
# fig.add_traces(Scatter(x=[s[i], s[i], e[i], e[i], s[i]], y=[l[i], h[i], h[i], l[i], l[i]], fill='toself'))


fig = go.Figure(data=go.Ohlc(x=bin_plt['bin_start'],
                    open=bin_plt['o'],
                    high=bin_plt['h'],
                    low=bin_plt['l'],
                    close=bin_plt['c']))

# go.Ohlc(x=bin_plt['Date'],
#                     open=df['AAPL.Open'],
#                     high=df['AAPL.High'],
#                     low=df['AAPL.Low'],
#                     close=df['AAPL.Close']))

#fig.add_traces(Scatter(x=df_plt['t'], y=(df_plt['bid'] + df_plt['ask'])/2, name='mid', mode="markers", marker={'size':2, 'color':'black'},showlegend=False))
fig.show()

In [9]:
bin_plt

,bin_start,bin_end,t0,t1,n_samples,o,h,l,c,twap
0,2023-10-02 18:26:00,2023-10-02 18:27:00,2023-10-02 18:26:22.617,2023-10-02 18:26:39.664,10,1.153965,1.154005,1.153965,1.153985,1.153984
1,2023-10-02 18:27:00,2023-10-02 18:28:00,2023-10-02 18:27:02.894,2023-10-02 18:27:54.783,24,1.153985,1.154025,1.153895,1.153895,1.153965
2,2023-10-02 18:28:00,2023-10-02 18:29:00,2023-10-02 18:28:00.052,2023-10-02 18:28:57.541,14,1.153905,1.153925,1.153885,1.153925,1.153900
3,2023-10-02 18:29:00,2023-10-02 18:30:00,2023-10-02 18:29:00.012,2023-10-02 18:29:44.678,14,1.153905,1.153925,1.153895,1.153905,1.153907
4,2023-10-02 18:30:00,2023-10-02 18:31:00,2023-10-02 18:30:00.828,2023-10-02 18:30:57.687,26,1.153925,1.153935,1.153765,1.153805,1.153819
5,2023-10-02 18:31:00,2023-10-02 18:32:00,2023-10-02 18:31:00.884,2023-10-02 18:31:50.058,30,1.153825,1.153835,1.153715,1.153725,1.153767
6,2023-10-02 18:32:00,2023-10-02 18:33:00,2023-10-02 18:32:00.813,2023-10-02 18:32:55.815,27,1.153715,1.153825,1.153715,1.153825,1.153778
7,2023-10-02 18:33:00,2023-10-02 18:34:00,2023-10-02 18:33:01.114,2023-10-02 18:33:58.282,22,1.153825,1.153825,1.153745,1.153745,1.153793
8,2023-10-02 18:34:00,2023-10-02 18:35:00,2023-10-02 18:34:12.622,2023-10-02 18:34:57.326,24,1.153755,1.153785,1.153745,1.153755,1.153759
9,2023-10-02 18:35:00,2023-10-02 18:36:00,2023-10-02 18:35:00.510,2023-10-02 18:35:59.330,30,1.153745,1.153795,1.153725,1.153775,1.153760
